In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
from collections import abc

import numpy as np

from harmonic_balance import arclength_continuation as alc
from harmonic_balance import continuation, freq, solve

ndarray = np.ndarray

## Define nonlinear force

In [ ]:
def get_slice(dof: int, N: int):
    return np.s_[dof * N : (dof + 1) * N]


def get_nonlinear(
    factor: float,
) -> tuple[
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
]:
    dof = 0
    s_ = lambda N: get_slice(dof, N)

    def f_nl(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * x[s_(N)] ** 3
        return res

    def df_nl_dx(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * 3 * x[s_(N)] ** 2
        return np.diag(res)

    def df_nl_d_xdot(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        shape = x.shape[0]
        return np.zeros((shape, shape), dtype=float)

    return f_nl, df_nl_dx, df_nl_d_xdot

## Define system

In [ ]:
n = 2

m1, m2 = 1, 2
c1, c2, c3 = 1, 0.5, 2
k1, k2, k3 = 1, 0.5, 2

M = np.diag([m1, m2])
C = np.array([[c1 + c2, -c2], [-c2, c2 + c3]])
K = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])

ks = (1, 1)
dofs = (0, 1)
is_cosines = (1, 0)
coefficients = (2, 1)

factor = 10
f_nl, df_nl_dx, df_nl_d_xdot = get_nonlinear(factor)

## Set harmonic balance parameters

In [ ]:
NH = 30
N = 128

b_ext = freq.get_b_ext(NH, n, ks, dofs, is_cosines, coefficients)

## Run harmonic balance

In [ ]:
s = 0.05

tol = 1e-11
max_iter = 100

omega_i0 = 2
A_i0 = freq.get_A(omega_i0, NH, M, C, K)
initial_guess = freq.solve_linear_system(A_i0, b_ext)
z_i0, *_ = solve.solve_nonlinear(
    omega_i0,
    initial_guess,
    A_i0,
    b_ext,
    f_nl,
    df_nl_dx,
    df_nl_d_xdot,
    NH,
    n,
    N,
    tol,
    max_iter=max_iter,
)

omega_i1 = omega_i0 + s
A_i1 = freq.get_A(omega_i1, NH, M, C, K)
initial_guess = freq.solve_linear_system(A_i1, b_ext)
z_i1, *_ = solve.solve_nonlinear(
    omega_i1,
    initial_guess,
    A_i0,
    b_ext,
    f_nl,
    df_nl_dx,
    df_nl_d_xdot,
    NH,
    n,
    N,
    tol,
    max_iter=max_iter,
)

## Run continuation

In [ ]:
alc_tol = 1e-11
alc_max_iter = 50

y_i0 = np.concat((z_i0, [omega_i0]))
y_i1 = np.concat((z_i1, [omega_i1]))
y_i2_k0 = alc.predict_y(y_i1, y_i0, s)

y_i2, rhs, converged, iters  = alc.correct_y(
    y_i2_k0,
    y_i1,
    b_ext,
    f_nl,
    df_nl_dx,
    df_nl_d_xdot,
    NH,
    n,
    N,
    s,
    M,
    C,
    K,
    alc_tol,
    alc_max_iter,
)

omega_i2, z_i2 = y_i2[-1].real, y_i2[:-1]

In [ ]:
print(f"norm(rhs) / norm(y_i2): {alc.get_rel_error(rhs, y_i2):.5e}")
print(f"iterations: {iters}")
print(f"norm(y_i2 - y_i1): {np.linalg.norm(y_i2 - y_i1):.15f}")
print(f"omega_i2: {omega_i2:.5f}")

norm(rhs) / norm(y_i2): 7.80394e-12
iterations: 42
norm(y_i2 - y_i1): 0.050000000000831
omega_i2: 2.06984


# Check shapes

In [ ]:
z_i1, z_i0 = b_ext.toarray() + 0.01, b_ext.toarray()
omega_i1, omega_i0 = 2.01, 2.0
y_i1, y_i0 = np.concat((z_i1, [omega_i1])), np.concat((z_i0, [omega_i0]))
s = 0.1

(
    alc.get_P(y_i1, y_i0, s),
    alc.get_dP_dz(z_i1, z_i0),
    alc.get_dP_d_omega(omega_i1, omega_i0),
)

(np.float64(-0.0037000000000000036),
 array([0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j]),
 0.019999999999999574)

In [ ]:
db_dz = solve.get_db_nl_dz(omega_i0, z_i1, df_nl_dx, df_nl_d_xdot, NH, n, N)
db_dz.shape

(62, 62)

In [ ]:
A = freq.get_A(omega_i0, NH, M, C, K)
(
    solve.get_R(z_i1, omega_i1, A, f_nl, b_ext, NH, n, N).shape,
    solve.get_dR_dz(A, db_dz).shape,
    continuation.get_dR_d_omega(z_i1, omega_i1, df_nl_d_xdot, NH, n, N, M, C).shape,
)

((62,), (62, 62), (62,))

In [ ]:
omega = 1.1
A = freq.get_A(omega, NH, M, C, K)
z_i0 = solve.get_initial_guess(A, b_ext)

z, *_ = solve.solve_nonlinear(
    omega, z_i0, A, b_ext, f_nl, df_nl_dx, df_nl_d_xdot, NH, n, N
)
z

array([-2.25694365e-17-2.98049650e-17j, -4.51388729e-18-5.96099300e-18j,
        2.13353362e-01-1.82337179e-01j, -1.69966533e-01-8.02034371e-02j,
        5.20625845e-17-3.69684861e-17j, -3.24643876e-18-7.88858121e-18j,
        9.43595250e-03-3.13047612e-02j, -2.47285610e-03-1.05383837e-03j,
       -1.99664882e-18-1.57022952e-17j, -9.41974199e-19+5.19618662e-20j,
       -2.13943557e-03-2.52272862e-03j, -1.23435135e-04+9.39237763e-05j,
       -3.97901200e-18-4.09192813e-18j, -1.64772239e-19+1.47222426e-19j,
       -3.31650958e-04+1.43632942e-05j,  8.86129533e-08+1.09526218e-05j,
        4.24924451e-19-5.65969249e-19j, -1.59019507e-20-1.27085305e-20j,
       -1.95073130e-05+2.74610015e-05j,  6.86843584e-07+5.15864392e-07j,
       -6.46723740e-19-1.19518265e-18j, -2.77229343e-20+1.41636376e-20j,
        9.84925428e-07+3.27988598e-06j,  6.86355338e-08-1.90221718e-08j,
        2.29323216e-19+3.60193484e-20j,  7.70911726e-22-4.35314993e-21j,
        3.24865259e-07+1.25779059e-07j,  2.31116302